# Interactive OpenAI Function Calling Notebook

Welcome to the Interactive Function Calling Notebook, tailored to guide you step-by-step through Open AI function. 

Creator: [Josh Bickett](https://twitter.com/josh_bickett)

Colab: [Interactive Notebook](https://colab.research.google.com/github/joshbickett/function-calling-notebook/blob/main/interactive-function-calling-notebook.ipynb)

Github: [Interactive Notebook](https://github.com/joshbickett/function-calling-notebook)

## What is a Function?

Functions let ChatGPT do things in the real world.



In [ ]:
!pip install openai

In [ ]:
import requests
import openai
import json

In [ ]:
openai.api_key = ''

In [ ]:

def launch_water_balloon(balloon_size):
    launch_results = f'Launching the {balloon_size} water balloon!'
    return launch_results



In [ ]:
functions = [
    {
        "name": "launch_water_balloon",
        "description": "This function launches a hypothetical water balloon",
        "parameters": {
            "type": "object",
            "properties": {
                "balloon_size": {
                    "type": "string",
                    "enum": ["Large", "Medium", "Small"],
                },
            },
            "required": ["location"],
        },
    }
]

available_functions = {
    "launch_water_balloon": launch_water_balloon,
}

In [ ]:

def run_conversation(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]
    print('[interactive-demo] Assistant', response["choices"][0]["message"])

    messages.append(response_message) 
    if response_message.get("function_call"):
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        print('[interactive-demo] second_response:', second_response)

In [ ]:
user_message = input("Please enter your message: ")
messages = [{"role": "user", "content": user_message}]
message_count = 0 
while message_count < 10: 
  run_conversation(messages)
  user_message = input("Please enter your message: ")
  new_user_message = {"role": "user", "content": user_message}
  messages = messages + [new_user_message]
  message_count += 1